# Ranking for property

In [235]:
import sys
sys.path.append("../repository/")
import database as db
import polars as pl
from math import floor

In [236]:
conn_wgi = db.DatabaseConnection("world-governance-index").connection
conn_ti = db.DatabaseConnection("transformation-index").connection
conn_ef = db.DatabaseConnection("economic-freedom").connection
conn_db = db.DatabaseConnection("doing-bussiness").connection

In [237]:
ti_freedom_right = pl.read_database(
    query="""
    SELECT country_id, no_interference_of_religious_dogmas
    FROM freedom
    WHERE year = 2020
    ORDER BY country_id                                        
    """,
    connection=conn_ti,
)
status = {
    (0, 3): "NOT FREE",
    (3, 6): "VERY INTOLERANT",
    (6, 9): "MODERATELY FREE",
    (9, 10): "MOSTLY FREE",
}

In [238]:
def get_status(val: float):
    res = None
    for k, v in status.items():
        if val in range(*k):
            res = v
    return res if res else "COMPLETELY FREE"

ti_freedom_right.with_columns(
    status=ti_freedom_right["no_interference_of_religious_dogmas"].map_elements(
        lambda val: get_status(val)
    )
)

country_id,no_interference_of_religious_dogmas,status
i64,f64,str
0,3.0,"""VERY INTOLERAN…"
2,10.0,"""COMPLETELY FRE…"
3,6.0,"""MODERATELY FRE…"
6,10.0,"""COMPLETELY FRE…"
10,10.0,"""COMPLETELY FRE…"
11,9.0,"""MOSTLY FREE"""
15,9.0,"""MOSTLY FREE"""
17,6.0,"""MODERATELY FRE…"
18,6.0,"""MODERATELY FRE…"
